In [11]:
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
import numpy_financial as npf


In [19]:
# Constants
annual_input = 20000
growth = 0.06
home_equity_growth = 0.03
income_growth = 0.035
interest_rate = 0.05625
current_home_price = 1_340_000
pct_down = 0.35
initial_income = 19500
initial_home_price = 2_000_000
initial_equity = 272_496.04
initial_principal = 1146.04
property_tax_rate = 0.014
months = 97  # 8 years

In [20]:
npf.pmt(interest_rate/12, 12*30, current_home_price*.8)

-6171.036652255405

In [31]:
npf.ipmt(interest_rate/12, 1, 12*30, current_home_price*.8)

array(-5025.)

In [32]:
npf.ppmt(interest_rate/12, 1, 12*30, current_home_price*.8)

-1146.036652255405

In [21]:


# Function to compute one month's values
def compute_month(month):
    year = (month - 1) // 12
    monthly_input = annual_input / 12 * ((1 + growth) ** year)
    
    # Cash input sum
    cash = sum(annual_input / 12 * ((1 + growth) ** ((m - 1) // 12)) for m in range(1, month + 1))
    cash_apprec = cash * ((1 + growth) ** (month / 12))
    
    # Appreciation and equity
    home_apprec = initial_home_price * ((1 + home_equity_growth) ** (month / 12) - 1)
    
    # Compound principal growth and contributions
    principal = initial_principal
    for m in range(1, month + 1):
        year_m = (m - 1) // 12
        contrib = annual_input / 12 * ((1 + growth) ** year_m)
        principal = principal * (1 + interest_rate / 12) + contrib

    equity = principal + home_apprec
    take_out = equity
    target_home_price = initial_home_price * ((1 + home_equity_growth) ** (month / 12))
    down_payment = target_home_price * pct_down
    income = initial_income * ((1 + income_growth) ** (month / 12))
    inc_req = income + monthly_input
    leftover = take_out - down_payment

    return {
    "year": year,
    "month": month,
    "cash in": round(cash, 2),
    "cash apprec": round(cash_apprec, 2),
    "home apprec": round(home_apprec, 2),
    "princ purchased": round(principal, 2),
    "equity": round(equity, 2),
    "take out": round(take_out, 2),
    "income": round(income, 2),
    "target home": round(target_home_price, 2),
    "down payment": round(down_payment, 2),
    "inc req": round(inc_req, 2),
    "leftover": round(leftover, 2)
}



In [22]:
with ThreadPoolExecutor() as executor:
    results = list(executor.map(compute_month, range(1, months + 1)))
df = pd.DataFrame(results)


In [23]:
df

,year,month,cash in,cash apprec,home apprec,princ purchased,equity,take out,income,target home,down payment,inc req,leftover
0,0,1,1666.67,1674.78,4932.54,2818.08,7750.62,7750.62,19555.98,2004932.54,701726.39,21222.65,-693975.77
1,0,2,3333.33,3365.86,9877.24,4497.96,14375.20,14375.20,19612.13,2009877.24,703457.04,21278.79,-689081.84
2,0,3,5000.00,5073.37,14834.14,6185.71,21019.85,21019.85,19668.43,2014834.14,705191.95,21335.10,-684172.10
3,0,4,6666.67,6797.42,19803.27,7881.37,27684.64,27684.64,19724.90,2019803.27,706931.14,21391.56,-679246.51
4,0,5,8333.33,8538.13,24784.65,9584.98,34369.63,34369.63,19781.52,2024784.65,708674.63,21448.19,-674305.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,7,93,190431.21,299129.04,514887.06,235941.64,750828.71,750828.71,25457.89,2514887.06,880210.47,27963.94,-129381.76
93,7,94,192937.26,304540.73,521089.45,239553.67,760643.13,760643.13,25530.97,2521089.45,882381.31,28037.02,-121738.18
94,7,95,195443.31,309998.01,527307.14,243182.63,770489.77,770489.77,25604.27,2527307.14,884557.50,28110.32,-114067.73
95,7,96,197949.36,315501.20,533540.16,246828.60,780368.76,780368.76,25677.78,2533540.16,886739.06,28183.83,-106370.30
